In [ ]:
!pip install --pre pycaret

In [20]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, r2_score
import seaborn as sns
from tqdm import tqdm
# 한글
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm , rc
plt.rc('font', family='NanumBarunGothic')


In [1]:
from pycaret.datasets import get_data
data = get_data('juice')

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


In [2]:
from pycaret.classification import *
s = setup(data, target = 'Purchase')

,Description,Value
0,Session id,994
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(748, 19)"
7,Transformed test set shape,"(322, 19)"
8,Ordinal features,1
9,Numeric features,17


In [34]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7041,0.7633,0.6274,0.7414,0.6795,0.4081,0.4132,0.5210
lightgbm,Light Gradient Boosting Machine,0.7040,0.7635,0.6240,0.7435,0.6782,0.4079,0.4136,0.2340
knn,K Neighbors Classifier,0.6761,0.7406,0.6572,0.6835,0.6696,0.3522,0.3529,0.2050
et,Extra Trees Classifier,0.6709,0.6911,0.7039,0.6606,0.6814,0.3417,0.3426,0.8260
dt,Decision Tree Classifier,0.6700,0.6336,0.7015,0.6602,0.6801,0.3399,0.3407,0.1100
rf,Random Forest Classifier,0.6688,0.7294,0.7089,0.6567,0.6817,0.3377,0.3389,1.4670
ada,Ada Boost Classifier,0.6559,0.7124,0.5381,0.7054,0.6093,0.3118,0.3216,0.2830
nb,Naive Bayes,0.5803,0.5853,0.5749,0.5816,0.5780,0.1606,0.1608,0.0770
qda,Quadratic Discriminant Analysis,0.5803,0.5853,0.5749,0.5816,0.5780,0.1606,0.1608,0.1040
svm,SVM - Linear Kernel,0.5766,0.0000,0.5725,0.5790,0.5733,0.1532,0.1543,0.0890


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [12]:
cd /content/drive/MyDrive/NICE 공모전

/content/drive/MyDrive/NICE 공모전


In [15]:
fin_mer = pd.read_csv('./data/재무데이터_액티브외감(0)_휴폐업(1)_머지.csv')
df_actice_naver = pd.read_csv('./data/네이버_데이터_트렌드_통합_액티브_외감_28979.csv')
df_deactice_naver = pd.read_csv('./data/네이버_데이터_트렌드_통합_휴폐업_6739.csv')
df_active_total = pd.read_excel('./data/액티브 중소법인 재무보유.xlsx',sheet_name= 2)
df_active_out = pd.read_excel('./data/액티브 중소법인 재무보유.xlsx', sheet_name=0)

In [16]:
df_actice_naver['target'] = '0'
df_deactice_naver['target'] = '1'
df_deactice_naver.head()

,company,BIZ_NO,mean,target
0,용진테크,1268145633,12.681016,1
1,신구건설,1388101749,20.780819,1
2,석정미디어,1068179240,14.178082,1
3,새한실업,2248114507,1.475237,1
4,석영엔터프라이즈,1168132785,27.671233,1


In [17]:
df_concat = pd.concat([df_actice_naver, df_deactice_naver])
df_concat.dropna(subset=['mean'], inplace=True)
df_concat.isna().sum()

company    0
BIZ_NO     0
mean       0
target     0
dtype: int64

In [21]:
scaler = MinMaxScaler()
df_concat['mean'] = scaler.fit_transform(df_concat[['mean']])
df_concat

,company,BIZ_NO,mean,target
0,한일가스산업,1138111739,0.183636,0
1,엠피아이,2078132193,0.626796,0
2,도영운수,1318167565,0.609217,0
3,중원산업,5048128251,0.561440,0
4,카타나골프,1098164260,0.401437,0
...,...,...,...,...
6734,게방식당팩토리,4808501629,0.098894,1
6735,우진푸드중부지점,8018501578,0.090627,1
6736,윤월드푸드코리아다크써클브로스,4118523419,0.027244,1
6737,기린테크,1738102533,0.327879,1


In [22]:
biz_list = df_active_total.BIZ_NO.unique()
df_concat_log = df_concat.copy()
df_concat_log = df_concat_log.reset_index()
del df_concat_log['index']
df_concat_log

,company,BIZ_NO,mean,target
0,한일가스산업,1138111739,0.183636,0
1,엠피아이,2078132193,0.626796,0
2,도영운수,1318167565,0.609217,0
3,중원산업,5048128251,0.561440,0
4,카타나골프,1098164260,0.401437,0
...,...,...,...,...
35712,게방식당팩토리,4808501629,0.098894,1
35713,우진푸드중부지점,8018501578,0.090627,1
35714,윤월드푸드코리아다크써클브로스,4118523419,0.027244,1
35715,기린테크,1738102533,0.327879,1


In [23]:
target_list = []
cnt = 0
print(df_concat_log.target.value_counts())
for idx in df_concat_log.index :
    if int(df_concat_log.loc[idx, 'BIZ_NO']) in list(biz_list):
        df_concat_log.loc[idx, 'target'] = '1'
        cnt += 1
print(len(biz_list), cnt)
df_concat_log.target.value_counts()

0    28978
1     6739
Name: target, dtype: int64
2083 362


0    28616
1     7101
Name: target, dtype: int64

In [26]:
df_concat_log_automl = df_concat_log[['mean', 'target']]

In [27]:
s = setup(df_concat_log_automl, target = 'target')

,Description,Value
0,Session id,7825
1,Target,target
2,Target type,Binary
3,Target mapping,"0: 0, 1: 1"
4,Original data shape,"(35717, 2)"
5,Transformed data shape,"(35717, 2)"
6,Transformed train set shape,"(25001, 2)"
7,Transformed test set shape,"(10716, 2)"
8,Numeric features,1
9,Preprocess,True


In [28]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8412,0.7675,0.2370,0.8681,0.3721,0.3137,0.4016,2.0300
knn,K Neighbors Classifier,0.8308,0.7276,0.3957,0.6228,0.4801,0.3857,0.4017,0.2930
lightgbm,Light Gradient Boosting Machine,0.8302,0.7567,0.2511,0.7052,0.3692,0.2963,0.3497,0.3650
ada,Ada Boost Classifier,0.8091,0.7095,0.0455,0.7247,0.0848,0.0677,0.1549,0.5980
et,Extra Trees Classifier,0.8018,0.7185,0.4196,0.5022,0.4569,0.3370,0.3392,1.2570
dt,Decision Tree Classifier,0.8016,0.7014,0.4210,0.5017,0.4575,0.3374,0.3395,0.1680
lr,Logistic Regression,0.8012,0.5900,0.0000,0.0000,0.0000,0.0000,0.0000,0.6940
nb,Naive Bayes,0.8012,0.5902,0.0000,0.0000,0.0000,0.0000,0.0000,0.1340
svm,SVM - Linear Kernel,0.8012,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1320
ridge,Ridge Classifier,0.8012,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1210


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [31]:
df_log_y = df_concat_log_automl[df_concat_log_automl.target =='1']
df_log_x = df_concat_log_automl[df_concat_log_automl.target =='0'].sample(len(df_log_y))

In [32]:
df_log_concat = pd.concat([df_log_y, df_log_x])
df_log_concat.target.value_counts()

1    7101
0    7101
Name: target, dtype: int64

In [33]:
s = setup(df_log_concat, target = 'target')
best = compare_models()

,Description,Value
0,Session id,522
1,Target,target
2,Target type,Binary
3,Target mapping,"0: 0, 1: 1"
4,Original data shape,"(14202, 2)"
5,Transformed data shape,"(14202, 2)"
6,Transformed train set shape,"(9941, 2)"
7,Transformed test set shape,"(4261, 2)"
8,Numeric features,1
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7041,0.7633,0.6274,0.7414,0.6795,0.4081,0.4132,0.5210
lightgbm,Light Gradient Boosting Machine,0.7040,0.7635,0.6240,0.7435,0.6782,0.4079,0.4136,0.1610
knn,K Neighbors Classifier,0.6761,0.7406,0.6572,0.6835,0.6696,0.3522,0.3529,0.2230
et,Extra Trees Classifier,0.6709,0.6911,0.7039,0.6606,0.6814,0.3417,0.3426,0.6460
dt,Decision Tree Classifier,0.6700,0.6336,0.7015,0.6602,0.6801,0.3399,0.3407,0.0860
rf,Random Forest Classifier,0.6688,0.7294,0.7089,0.6567,0.6817,0.3377,0.3389,1.0230
ada,Ada Boost Classifier,0.6559,0.7124,0.5381,0.7054,0.6093,0.3118,0.3216,0.2800
nb,Naive Bayes,0.5803,0.5853,0.5749,0.5816,0.5780,0.1606,0.1608,0.0750
qda,Quadratic Discriminant Analysis,0.5803,0.5853,0.5749,0.5816,0.5780,0.1606,0.1608,0.0800
svm,SVM - Linear Kernel,0.5766,0.0000,0.5725,0.5790,0.5733,0.1532,0.1543,0.0740


Processing:   0%|          | 0/61 [00:00<?, ?it/s]